<a href="https://colab.research.google.com/github/MehranBehzadi/LLM-project/blob/main/LLM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch

In [ ]:
pip install nltk spacy gensim

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
ds['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
ds['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [ ]:
import pandas as pd
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [ ]:
ds_train.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [ ]:
ds_test.head()

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(ds_train)
test_dataset = Dataset.from_pandas(ds_test)

new_ds = DatasetDict(
    {
        'train': train_dataset,
        'test': test_dataset
    }
)

new_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=True, stop_words="english")

# Fit and transform the training data
train_bow = vectorizer.fit_transform(new_ds['train']['text'])

# Transform the test data
test_bow = vectorizer.transform(new_ds['test']['text'])


In [ ]:
vectorizer = CountVectorizer(lowercase=True, stop_words="english", max_features=10000, min_df=2, max_df=0.95)


In [ ]:
train_labels = new_ds['train']['label']
test_labels = new_ds['test']['label']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(max_iter=1000)
model.fit(train_bow, train_labels)

# Predict on test set
test_predictions = model.predict(test_bow)

# Evaluate
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy:", accuracy)
print(classification_report(test_labels, test_predictions))

Accuracy: 0.85884
              precision    recall  f1-score   support

           0       0.85      0.87      0.86     12500
           1       0.87      0.85      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000



In [ ]:
# Extract the 'text' column from the training and test datasets
train_texts = new_ds['train']['text']
test_texts = new_ds['test']['text']

# Display the first few examples to confirm
print("Training Texts:", train_texts[:5])
print("Test Texts:", test_texts[:5])


Training Texts: ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few an

In [ ]:
import random

# Randomly select 500 texts
subset_texts = random.sample(train_texts, 500)

In [ ]:
pip install transformers

In [ ]:
from transformers import pipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
pipe = pipeline("text-classification", model="lvwerra/distilbert-imdb")

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
results = pipe(subset_texts, truncation=True, max_length=512)

In [ ]:
results

[{'label': 'NEGATIVE', 'score': 0.996791422367096},
 {'label': 'POSITIVE', 'score': 0.9868740439414978},
 {'label': 'POSITIVE', 'score': 0.9375452995300293},
 {'label': 'POSITIVE', 'score': 0.9800201654434204},
 {'label': 'NEGATIVE', 'score': 0.9216769933700562},
 {'label': 'NEGATIVE', 'score': 0.9847334027290344},
 {'label': 'NEGATIVE', 'score': 0.996142566204071},
 {'label': 'NEGATIVE', 'score': 0.9889543056488037},
 {'label': 'NEGATIVE', 'score': 0.9792677164077759},
 {'label': 'POSITIVE', 'score': 0.9887079000473022},
 {'label': 'POSITIVE', 'score': 0.981143593788147},
 {'label': 'NEGATIVE', 'score': 0.8888429999351501},
 {'label': 'POSITIVE', 'score': 0.9955981373786926},
 {'label': 'POSITIVE', 'score': 0.9826927185058594},
 {'label': 'NEGATIVE', 'score': 0.9915225505828857},
 {'label': 'POSITIVE', 'score': 0.9957691431045532},
 {'label': 'NEGATIVE', 'score': 0.9928416013717651},
 {'label': 'POSITIVE', 'score': 0.9876164793968201},
 {'label': 'NEGATIVE', 'score': 0.987305879592895

In [ ]:
# Extract labels and scores
predicted_labels = [result["label"] for result in results]
predicted_scores = [result["score"] for result in results]

# Display some results
for text, label, score in zip(subset_texts[:5], predicted_labels[:5], predicted_scores[:5]):
    print(f"Text: {text}\nPredicted Label: {label}, Confidence: {score:.2f}\n")


Text: For shame, for shame that a fine actor such as Joseph Fiennes would allow himself to be cast in this piece of nauseating drivel. The movie was not only bad, but down right horrible and of no redeeming quality. The plot, (was there one?) seemed to go no where. The Russians played silly kill or be killed games and the rest of the cast should be declared null and void for their pathetic performances. I gave up about 3/4 of the way through and turned it off. A "1" for awful only because there is nothing lower. Don't waste your time on this one, you'll not miss anything.
Predicted Label: NEGATIVE, Confidence: 1.00

Text: This is one of the better comedies that has ever been on television. Season one was hilarious as were most of the following seasons. The only reason that I give this show a 9/10 is because of the unfortunate final season. The only good part of the final season was the finale. My favorite part of this show was the scenes that cut to people's imaginations, often depicti

In [ ]:
pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
 train_ds = new_ds['train']  # Assign the 'train' split to train_ds
 eval_ds = new_ds['test']  # Assign the 'test' split to eval_ds for evaluation

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

In [ ]:
train_ds = train_ds.map(preprocess_function, batched=True)
eval_ds = eval_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
print(train_ds.column_names)

['text', 'label', 'input_ids', 'attention_mask']


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load the pre-trained DistilBERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("lvwerra/distilbert-imdb")


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1) # Get predicted classes
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir='my_model',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    push_to_hub=False,
)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-33-92b92ef0a545>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.147300
1000,0.143900
1500,0.135300


TrainOutput(global_step=1563, training_loss=0.1424872226922541, metrics={'train_runtime': 1328.5631, 'train_samples_per_second': 18.817, 'train_steps_per_second': 1.176, 'total_flos': 3311684966400000.0, 'train_loss': 0.1424872226922541, 'epoch': 1.0})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
repo_name = "LLM-project"
training_args = TrainingArguments(
   output_dir=repo_name,
   push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,  # Use the new training_args here
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-43-26f8ede3f77a>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.evaluate()

{'eval_loss': 0.24149523675441742,
 'eval_model_preparation_time': 0.0029,
 'eval_accuracy': 0.92868,
 'eval_runtime': 380.7321,
 'eval_samples_per_second': 65.663,
 'eval_steps_per_second': 8.208}

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1736046762.3db3b60b2e32.500.1:   0%|          | 0.00/409 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mehranbhz/LLM-project/commit/a508cb1d9871a46407064a6429faeb9afab2b155', commit_message='End of training', commit_description='', oid='a508cb1d9871a46407064a6429faeb9afab2b155', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mehranbhz/LLM-project', endpoint='https://huggingface.co', repo_type='model', repo_id='mehranbhz/LLM-project'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline
data = [ ] # Replace with your list of text data
my_model = pipeline(model="mehranbhz/LLM-project")
my_model(data)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0


[]

# New Section